In [35]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import warnings as wn
wn.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
import pickle

In [61]:
raw_liver=pd.read_csv("Indian Liver Patient Dataset (ILPD).csv")
raw_liver.columns=['Age','Gender','Total_Bilirubin','Direct_Bilirubin','Alkaline_Phosphotase','Alamine_Aminotransferase','Aspartate_Aminotransferase','Total_Protiens','Albumin','Albumin_and_Globulin_Ratio','Target']
col=['Age', 'Total_Bilirubin', 'Direct_Bilirubin', 'Alkaline_Phosphotase',
       'Alamine_Aminotransferase', 'Aspartate_Aminotransferase',
       'Total_Protiens', 'Albumin',
       'Albumin_and_Globulin_Ratio', 'Gender','Target']
data=pd.DataFrame()
for x in col:
    data[x]=raw_liver[x]

In [37]:
col=['Age', 'Total_Bilirubin', 'Direct_Bilirubin', 'Alkaline_Phosphotase',
       'Alamine_Aminotransferase', 'Aspartate_Aminotransferase',
       'Total_Protiens', 'Albumin',
       'Albumin_and_Globulin_Ratio', 'Gender']
data=pd.DataFrame()
for x in col:
    data[x]=raw_liver[x]

In [38]:
def NAN_replace(df,column_name,x):
    #x=mean,median,mode,random,nan_binary,end_distribution
    if x=='mean':
        df[column_name+'_'+x+'_replaced']=df[column_name]
        df[column_name+'_'+x+'_replaced']=df[column_name+'_'+x+'_replaced'].replace(to_replace=np.nan,value=df[column_name].mean(),inplace=False)
    if x=='median':
        df[column_name+'_'+x+'_replaced']=df[column_name]
        df[column_name+'_'+x+'_replaced']=df[column_name+'_'+x+'_replaced'].replace(to_replace=np.nan,value=df[column_name].median(),inplace=False)
    if x=='mode':
        df[column_name+'_'+x+'_replaced']=df[column_name]
        df[column_name+'_'+x+'_replaced']=df[column_name+'_'+x+'_replaced'].replace(to_replace=np.nan,value=df[column_name].mode(),inplace=False)
    if x=='random':
        df[column_name+'_random'] = df[column_name]  
        random_sample = df[column_name].dropna().sample(df[column_name].isnull().sum(), random_state=0)  
        random_sample.index = df[df['Age'].isnull()].index  
        df.loc[df2[column_name].isnull(), column_name+'_random'] = random_sample 
    if x=='nan_binary':
        df[column_name+'_nan_binary']=np.where(d[column_name].isnull(),1,0)
    if x=='end_distribution':
        df[column_name+'_end _distribution']=df[column_name].fillna(df[column_name].mean()+3*df3[column_name].std())
    

In [39]:
NAN_replace(data,'Albumin_and_Globulin_Ratio','median')

In [40]:
data['Gender']=data['Gender'].replace({'Male':1,'Female':0})

In [41]:
data.drop(columns='Albumin_and_Globulin_Ratio',inplace=True)

In [42]:
x=data
y=raw_liver['Target']
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy=train_test_split(x,y,test_size=0.25,random_state=47)
print(trainx.shape  ,  trainy.shape)
print(testx.shape  ,   testy.shape)

(436, 10) (436,)
(146, 10) (146,)


In [43]:
from imblearn.combine import SMOTETomek
sm=SMOTETomek()
smot_trainx,smot_trainy=sm.fit_resample(trainx,trainy)
print(trainy.value_counts())
print(smot_trainy.value_counts())

Target
1    312
2    124
Name: count, dtype: int64
Target
2    298
1    298
Name: count, dtype: int64


In [44]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
smot_scl_trainx=ss.fit_transform(smot_trainx)
scl_testx=ss.transform(testx)

In [45]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score,confusion_matrix,roc_auc_score
model=GradientBoostingClassifier(random_state=47)
model.fit(smot_scl_trainx,smot_trainy)
ypred_svc=model.predict(scl_testx)

print('f1_score        ',f1_score(ypred_svc,testy))
print('accuracy_score  ',accuracy_score(ypred_svc,testy))
print('precision_score ',precision_score(ypred_svc,testy))
print('recall_score    ',recall_score(ypred_svc,testy))
print('classification_report')
print(classification_report(ypred_svc,testy))
print('confusion_matrix')
print(confusion_matrix(ypred_svc,testy))

f1_score         0.6875
accuracy_score   0.589041095890411
precision_score  0.6407766990291263
recall_score     0.7415730337078652
classification_report
              precision    recall  f1-score   support

           1       0.64      0.74      0.69        89
           2       0.47      0.35      0.40        57

    accuracy                           0.59       146
   macro avg       0.55      0.55      0.54       146
weighted avg       0.57      0.59      0.58       146

confusion_matrix
[[66 23]
 [37 20]]


In [46]:
from sklearn.model_selection import RandomizedSearchCV
grid_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0],
    'max_features': ['auto', 'sqrt', 'log2', None]
}
modelcv=RandomizedSearchCV(estimator=model,param_distributions=grid_gb,n_iter=5,n_jobs=-1,cv=5,random_state=47)
modelcv.fit(smot_scl_trainx,smot_trainy)
print(modelcv.best_params_)




{'subsample': 1.0, 'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 5, 'learning_rate': 1}


In [47]:
modelcv=GradientBoostingClassifier(subsample= 1, n_estimators= 300, min_samples_split= 10,
                                   min_samples_leaf= 1, max_features= 'sqrt', max_depth= 5, learning_rate= 1,random_state=47)
modelcv.fit(smot_scl_trainx,smot_trainy)
ypred_dt=modelcv.predict(scl_testx)

print('f1_score        ',f1_score(ypred_dt,testy))
print('accuracy_score  ',accuracy_score(ypred_dt,testy))
print('precision_score ',precision_score(ypred_dt,testy))
print('recall_score    ',recall_score(ypred_dt,testy))
print('classification_report')
print(classification_report(ypred_dt,testy))
print('confusion_matrix')
print(confusion_matrix(ypred_dt,testy))

f1_score         0.7263681592039801
accuracy_score   0.6232876712328768
precision_score  0.7087378640776699
recall_score     0.7448979591836735
classification_report
              precision    recall  f1-score   support

           1       0.71      0.74      0.73        98
           2       0.42      0.38      0.40        48

    accuracy                           0.62       146
   macro avg       0.56      0.56      0.56       146
weighted avg       0.61      0.62      0.62       146

confusion_matrix
[[73 25]
 [30 18]]


In [59]:
pickle.dump(ss, open('StandardScaler.pkl','wb'))
pickle.dump(modelcv, open('model.pkl','wb'))

In [57]:

model = pickle.load(open('model.pkl','rb'))
print(model.predict([[62,1.0,0.4,182,14,20,6.8,3.4,1,0.74]]))

[1]


In [58]:
data.columns

Index(['Age', 'Total_Bilirubin', 'Direct_Bilirubin', 'Alkaline_Phosphotase',
       'Alamine_Aminotransferase', 'Aspartate_Aminotransferase',
       'Total_Protiens', 'Albumin', 'Gender',
       'Albumin_and_Globulin_Ratio_median_replaced'],
      dtype='object')

In [66]:
data[data['Target']==2]

,Age,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Gender,Target
7,17,0.9,0.3,202,22,19,7.4,4.1,1.20,Male,2
11,64,0.9,0.3,310,61,58,7.0,3.4,0.90,Male,2
14,25,0.6,0.1,183,91,53,5.5,2.3,0.70,Male,2
16,33,1.6,0.5,165,15,23,7.3,3.5,0.92,Male,2
23,63,0.9,0.2,194,52,45,6.0,3.9,1.85,Male,2
...,...,...,...,...,...,...,...,...,...,...,...
550,29,1.2,0.4,160,20,22,6.2,3.0,0.90,Male,2
563,38,0.6,0.1,165,22,34,5.9,2.9,0.90,Female,2
565,50,1.0,0.3,191,22,31,7.8,4.0,1.00,Female,2
577,60,0.5,0.1,500,20,34,5.9,1.6,0.37,Male,2
